---

### <center> <b> <font color = 'green'> Aplicação de kernels </font> <center>  

### <center> <b> <font color = 'white'>🌳Convolutional Neural Networks - CNNs🌳 </font> <center> 

---

In [ ]:
# Função para aplicar diferentes filtros kernels
# Nesta versão anterior da função eu convertia a imagem para RGB e aplicava o filtro canal a canal — isso mantém as 3 camadas (R, G, B) e o resultado acaba “colorido”.
import numpy as np
from scipy.signal import convolve2d
from PIL import Image, ImageDraw, ImageFont

def aplicar_filtro_com_titulo(imagem, kernel, titulo="Título da Imagem"):
    """
    Aplica um filtro à imagem (RGB ou tons de cinza),
    adiciona uma borda superior e escreve um título centralizado.
    """
    # Converter imagem PIL para array NumPy
    if isinstance(imagem, Image.Image):
        img_arr = np.array(imagem.convert("RGB"))  # força RGB (3 canais)
    else:
        img_arr = imagem

    # Se for imagem colorida (RGB)
    if len(img_arr.shape) == 3 and img_arr.shape[2] == 3:
        canais = []
        for i in range(3):
            canal = convolve2d(img_arr[:, :, i], kernel, mode='same', boundary='symm')
            canal = np.clip(canal, 0, 255).astype(np.uint8)
            canais.append(canal)
        filtrada = np.stack(canais, axis=2)
        img_final = Image.fromarray(filtrada, mode="RGB")
    else:
        # Se for imagem a tons de cinza
        filtrada = convolve2d(img_arr, kernel, mode='same', boundary='symm')
        filtrada = np.clip(filtrada, 0, 255).astype(np.uint8)
        img_final = Image.fromarray(filtrada, mode="L")

    # Criar borda superior preta
    largura, altura = img_final.size
    altura_borda = 50
    nova_imagem = Image.new(img_final.mode, (largura, altura + altura_borda), color=0)
    nova_imagem.paste(img_final, (0, altura_borda))

    # Escrever título centralizado
    if titulo:
        draw = ImageDraw.Draw(nova_imagem)
        try:
            font = ImageFont.truetype("arial.ttf", 24)
        except Exception:
            font = ImageFont.load_default()

        # Pillow >= 10 usa textbbox
        if hasattr(draw, "textbbox"):
            left, top, right, bottom = draw.textbbox((0, 0), titulo, font=font)
            text_w, text_h = right - left, bottom - top
        else:
            text_w, text_h = draw.textsize(titulo, font=font)

        pos = ((largura - text_w) // 2, (altura_borda - text_h) // 2)
        cor_texto = 255 if nova_imagem.mode == "L" else (255, 255, 255)
        draw.text(pos, titulo, fill=cor_texto, font=font)

    return nova_imagem



In [ ]:
# Dicionário de Filtros Kernel
import numpy as np

filtros = {
    "sobel_horizontal": np.array([[-1, -2, -1],
                                  [ 0,  0,  0],
                                  [ 1,  2,  1]]),

    "sobel_vertical": np.array([[-1,  0,  1],
                                [-2,  0,  2],
                                [-1,  0,  1]]),

    "linhas_horizontal": np.array([[-1, -1, -1],
                                   [ 2,  2,  2],
                                   [-1, -1, -1]]),

    "linhas_vertical": np.array([[-1,  2, -1],
                                 [-1,  2, -1],
                                 [-1,  2, -1]]),

    "gaussian_blur": (1/16) * np.array([[1, 2, 1],
                                        [2, 4, 2],
                                        [1, 2, 1]]),

    "laplaciano": np.array([[ 0, -1,  0],
                            [-1,  4, -1],
                            [ 0, -1,  0]]),

    "detalhamento": np.array([[ 0, -1,  0],
                              [-1,  5, -1],
                              [ 0, -1,  0]]),

    "cantos": np.array([[-1, -1, -1],
                        [-1,  8, -1],
                        [-1, -1, -1]]),

    "curvas": np.array([[ 0,  1,  0],
                        [ 1, -4,  1],
                        [ 0,  1,  0]]),

    "bordas_diagonais": np.array([[ 1,  0, -1],
                                  [ 0,  0,  0],
                                  [-1, 0,  1]]),

    "prewitt_horizontal": np.array([[-1, -1, -1],
                                    [ 0,  0,  0],
                                    [ 1,  1,  1]]),

    "prewitt_vertical": np.array([[-1, 0, 1],
                                  [-1, 0, 1],
                                  [-1, 0, 1]])
}


In [ ]:
from PIL import Image

# Abrir imagem
img = Image.open("image.png")

# Aplicar filtro Laplaciano com título
resultado = aplicar_filtro_com_titulo(img, filtros['laplaciano'], titulo="Filtro Laplaciano")
resultado2 = aplicar_filtro_com_titulo(img, filtros['prewitt_horizontal'], titulo="Filtro Prewitt (horizontal)")
resultado3 = aplicar_filtro_com_titulo(img, filtros['sobel_vertical'], titulo="Filtro Sobel (vertical)")

# Mostrar resultado
#resultado.show()

# (Opcional) Guardar a imagem
resultado.save("Laplaciano.png")
resultado2.save("Prewitt_H.png")
resultado3.save("Sobel-V.png")


In [15]:
# Função para aplicar diferentes filtros kernels
# Aqui para obter o resultado em preto e branco (escala de cinza), a solução é simples: convertemos logo a imagem para tons de cinza antes de aplicar o filtro.
import numpy as np
from scipy.signal import convolve2d
from PIL import Image, ImageDraw, ImageFont

def aplicar_filtro_com_titulo(imagem, filtro_nome, titulo=None):
    """
    Aplica um filtro pré-definido à imagem (sempre em tons de cinza),
    adiciona uma borda superior e escreve um título centralizado.
    """
    # Dicionário de filtros
    filtros = {
        "sobel_horizontal": np.array([[-1, -2, -1],
                                      [ 0,  0,  0],
                                      [ 1,  2,  1]]),

        "sobel_vertical": np.array([[-1,  0,  1],
                                    [-2,  0,  2],
                                    [-1,  0,  1]]),

        "linhas_horizontal": np.array([[-1, -1, -1],
                                       [ 2,  2,  2],
                                       [-1, -1, -1]]),

        "linhas_vertical": np.array([[-1,  2, -1],
                                     [-1,  2, -1],
                                     [-1,  2, -1]]),

        "gaussian_blur": (1/16) * np.array([[1, 2, 1],
                                            [2, 4, 2],
                                            [1, 2, 1]]),

        "laplaciano": np.array([[ 0, -1,  0],
                                [-1,  4, -1],
                                [ 0, -1,  0]]),

        "detalhamento": np.array([[ 0, -1,  0],
                                  [-1,  5, -1],
                                  [ 0, -1,  0]]),

        "cantos": np.array([[-1, -1, -1],
                            [-1,  8, -1],
                            [-1, -1, -1]]),

        "curvas": np.array([[ 0,  1,  0],
                            [ 1, -4,  1],
                            [ 0,  1,  0]]),

        "bordas_diagonais": np.array([[ 1,  0, -1],
                                      [ 0,  0,  0],
                                      [-1, 0,  1]]),

        "prewitt_horizontal": np.array([[-1, -1, -1],
                                        [ 0,  0,  0],
                                        [ 1,  1,  1]]),

        "prewitt_vertical": np.array([[-1, 0, 1],
                                      [-1, 0, 1],
                                      [-1, 0, 1]])
    }

    if filtro_nome not in filtros:
        raise ValueError(f"Filtro '{filtro_nome}' não encontrado. Filtros disponíveis: {list(filtros.keys())}")

    kernel = filtros[filtro_nome]

    # Converter imagem sempre para tons de cinza (2D)
    if isinstance(imagem, Image.Image):
        img_arr = np.array(imagem.convert("L"))
    else:
        img_arr = imagem

    # Aplicar filtro
    filtrada = convolve2d(img_arr, kernel, mode='same', boundary='symm')
    filtrada = np.clip(filtrada, 0, 255).astype(np.uint8)
    img_final = Image.fromarray(filtrada, mode="L")

    # Criar borda preta no topo
    largura, altura = img_final.size
    altura_borda = 50
    nova_imagem = Image.new("L", (largura, altura + altura_borda), color=0)
    nova_imagem.paste(img_final, (0, altura_borda))

    # Texto
    if titulo is None:
        titulo = f"Filtro: {filtro_nome}"

    draw = ImageDraw.Draw(nova_imagem)
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except Exception:
        font = ImageFont.load_default()

    if hasattr(draw, "textbbox"):
        left, top, right, bottom = draw.textbbox((0, 0), titulo, font=font)
        text_w, text_h = right - left, bottom - top
    else:
        text_w, text_h = draw.textsize(titulo, font=font)

    pos = ((largura - text_w) // 2, (altura_borda - text_h) // 2)
    draw.text(pos, titulo, fill=255, font=font)

    return nova_imagem


In [17]:
from PIL import Image

# Abrir imagem
img = Image.open("image.png")

# Aplicar filtro Laplaciano (preto e branco)
resultado = aplicar_filtro_com_titulo(img, "laplaciano", titulo= "Filtro Laplaciano")
resultado2 = aplicar_filtro_com_titulo(img, "prewitt_horizontal", titulo= "Filtro Prewitt (horizontal)")
resultado3 = aplicar_filtro_com_titulo(img, "sobel_vertical", titulo= "Filtro Sobel (vertical)")
resultado4 = aplicar_filtro_com_titulo(img, "gaussian_blur", titulo= "Filtro Gaussian Blur")


# Mostrar resultado
#resultado.show()

# Mostrar
resultado.show()

# Guardar
resultado.save("Laplaciano.png")
resultado2.save("Prewitt_H.png")
resultado3.save("Sobel-V.png")
resultado4.save("gaussian_blur.png")

C:\Users\DEIVISON VENICIO\AppData\Local\Temp\ipykernel_12768\4074541049.py:77: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img_final = Image.fromarray(filtrada, mode="L")


In [ ]:
filtros

## **The End------------------------**